<font face="Century Gothic" color='#0F3FA05' size="20px"><h1><left>Flood Disaster Assistance:</left></h1></font> <font face="Century Gothic" color='#26A3DF' size="18px"><left>Multi-Agent AI System</left></font>

This system implements a multi-agent architecture using CrewAI to provide real-time flood disaster assistance. It coordinates between different specialized agents to provide comprehensive emergency response, resource location, and insurance guidance.

# `Main components:`

1. Multiple specialized agents (Coordinator, Resource Locator, Insurance, Emergency)
2. Task management and coordination system
3. Data models for structured information handling
4. Mock implementations of external services (replaceable with real APIs)

# System Architecture Overview:

1. Coordinator Agent: Manages the overall flow and delegates tasks to specialized agents
2. Resource Locator Agent: Finds and validates real-time resource availability
3. Insurance Specialist Agent: Handles insurance-related queries and processes
4. Emergency Response Agent: Provides immediate assistance information
5. Documentation Agent: Manages and provides process documentation

#Key components and design decisions:

##Architecture Overview:


Uses CrewAI framework for orchestrating multiple specialized agents
Implements 4 core agents: Coordinator, Resource Locator, Insurance Specialist, and Emergency Response
Uses Pydantic models for strong typing and data validation
Implements custom tools for resource and insurance information retrieval


##Key Features:


- Real-time resource availability tracking
- Insurance information and claims process guidance
- Emergency response coordination
- Location-based assistance
- Support for both customers and non-customers


##Agent Roles:


- Coordinator Agent: Orchestrates overall response
- Resource Agent: Manages real-time resource availability
- Insurance Agent: Handles insurance-related queries
- Emergency Agent: Provides immediate safety guidance


Implementation Notes:


- The system is designed to be easily extensible
- Includes mock implementations that can be replaced with real API calls
- Uses Pydantic for data validation and structure
- Implements sequential processing for coordinated response

In [1]:
!pip install crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from typing import List, Dict
from datetime import datetime
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
import dotenv

In [3]:
from google.colab import userdata, files # For accessing Colab secrets

In [4]:
# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

Data Models
==========
These Pydantic models define the structure for various data types used in the system to provide type validation and serialization capabilites.

Represents a geographic location with address information.

Attributes:
- latitude (float): The latitude coordinate
- longitude (float): The longitude coordinate
- address (str): The street address
- zip_code (str): The postal/ZIP code

In [5]:
# Define data models
class Location(BaseModel):
    latitude: float
    longitude: float
    address: str
    zip_code: str

class ResourceInfo(BaseModel):
    name: str
    type: str
    location: Location
    availability: bool
    contact: str
    last_updated: datetime

class InsuranceInfo(BaseModel):
    company: str
    policy_types: List[str]
    contact: str
    claim_process: str
    online_portal: str

Language Model Configuration: Initializes the LLM used by all agents.

In [6]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0.7,
    verbose=True
)

Agent Definitions:
- Defines the specialized agents that form the core of the system.
- Each agent has specific roles, goals, and capabilities.

In [7]:
# Define Agents
coordinator_agent = Agent(
    role='Coordination Specialist',
    goal='Efficiently coordinate emergency response and resource allocation',
    backstory="""You are an experienced emergency response coordinator with expertise
    in managing flood disaster situations. Your primary responsibility is to ensure
    all assistance requests are handled efficiently and effectively.""",
    allow_delegation=True,
    verbose=True,
    llm=llm
)

resource_agent = Agent(
    role='Resource Locator',
    goal='Identify and verify available emergency resources',
    backstory="""You are a resource management expert specializing in emergency
    situations. You maintain real-time information about available shelters,
    supplies, and emergency services.""",
    verbose=True,
    llm=llm
)

insurance_agent = Agent(
    role='Insurance Specialist',
    goal='Provide insurance-related guidance and support',
    backstory="""You are an insurance expert with deep knowledge of flood insurance
    policies, claims processes, and emergency procedures. You help both customers
    and non-customers navigate insurance-related aspects of flood emergencies.""",
    verbose=True,
    llm=llm
)

emergency_agent = Agent(
    role='Emergency Response Specialist',
    goal='Provide immediate emergency guidance and assistance',
    backstory="""You are an emergency response expert specialized in flood situations.
    You provide critical guidance for immediate safety and assistance needs.""",
    verbose=True,
    llm=llm
)



Main system implementation:
- The FloodAssistanceSystem class orchestrates the entire system,
managing agents, tasks, and response generation

Helper functions:
- Utility functions for fetching and processing data from external services.
- in production - these would be replaced with actual API calls.

In [8]:
class FloodAssistanceSystem:
    def __init__(self):
        self.crew = None
        self.location = None

    def initialize_response(self, location: Location):
        """Initialize the emergency response system for a given location"""
        self.location = location

        tasks = [
            Task(
                description=f"""Assess the flood situation at {location.address} and coordinate
                initial response efforts. Determine priority needs and available resources.
                Consider the following factors:
                1. Current flood severity
                2. Number of people affected
                3. Immediate safety concerns""",
                agent=coordinator_agent,
                expected_output="""Detailed assessment report including:
                - Severity level
                - Number of affected individuals
                - Priority needs
                - Available resources
                - Initial response plan"""
            ),
            Task(
                description=f"""Locate and verify all available emergency resources near
                {location.address}. Provide real-time availability updates for:
                - Emergency shelters
                - Medical facilities
                - Supply distribution centers""",
                agent=resource_agent,
                expected_output="""List of available resources including:
                - Location and contact information
                - Current availability status
                - Capacity and capabilities
                - Special services offered"""
            ),
            Task(
                description=f"""Provide insurance guidance for the affected area ({location.zip_code}).
                Include information about:
                - Emergency claim procedures
                - Required documentation
                - Contact information
                - Online resources""",
                agent=insurance_agent,
                expected_output="""Comprehensive insurance guidance including:
                - Claims process steps
                - Required documentation checklist
                - Contact information for claims
                - Online resources and portals
                - Special disaster provisions"""
            ),
            Task(
                description=f"""Develop immediate emergency response plan for {location.address}.
                Include:
                - Evacuation routes
                - Safety procedures
                - Emergency contacts
                - Real-time updates""",
                agent=emergency_agent,
                expected_output="""Detailed emergency response plan including:
                - Evacuation routes and procedures
                - Safety guidelines
                - Emergency contact list
                - Communication protocols
                - Real-time status updates"""
            )
        ]

        self.crew = Crew(
            agents=[coordinator_agent, resource_agent, insurance_agent, emergency_agent],
            tasks=tasks,
            verbose=True
        )

    def get_assistance(self) -> Dict:
        """Execute the assistance process and return comprehensive response"""
        if not self.crew:
            raise ValueError("System not initialized. Call initialize_response first.")

        result = self.crew.kickoff()
        return self._process_results(result)

    def _process_results(self, crew_result: str) -> Dict:
        """Process and structure the results from the crew's execution"""
        return {
            "emergency_resources": get_local_resources(self.location),
            "insurance_info": get_insurance_info(self.location.zip_code),
            "response_plan": crew_result
        }

def get_local_resources(location: Location) -> List[ResourceInfo]:
    """Fetch real-time local emergency resources based on location"""
    # Mock implementation - replace with actual API calls in production
    return [
        ResourceInfo(
            name="Emergency Shelter Alpha",
            type="shelter",
            location=location,
            availability=True,
            contact="555-0123",
            last_updated=datetime.now()
        )
    ]

def get_insurance_info(zip_code: str) -> List[InsuranceInfo]:
    """Fetch insurance company information for a given area"""
    # Mock implementation - replace with actual API calls in production
    return [
        InsuranceInfo(
            company="SafeGuard Insurance",
            policy_types=["Flood", "Property"],
            contact="1-800-SAFEGUARD",
            claim_process="Online or Phone",
            online_portal="https://safeguard.com/claims"
        )
    ]

Main execution - example implementation showing how to use the system.

In [9]:
def main():
    """Example usage of the Flood Assistance System"""
    # Initialize the system with a sample location
    location = Location(
        latitude=40.7128,
        longitude=-74.0060,
        address="123 Emergency St, New York, NY",
        zip_code="10001"
    )

    # Create and initialize the system
    system = FloodAssistanceSystem()
    system.initialize_response(location)

    # Get comprehensive assistance
    try:
        assistance = system.get_assistance()

        # Display results
        print("\nEmergency Resources:")
        for resource in assistance["emergency_resources"]:
            print(f"- {resource.name}: {resource.contact}")

        print("\nInsurance Information:")
        for insurance in assistance["insurance_info"]:
            print(f"- {insurance.company}: {insurance.contact}")

        print("\nResponse Plan:")
        print(assistance["response_plan"])

    except Exception as e:
        print(f"Error occurred: {str(e)}")

In [10]:
if __name__ == "__main__":
    main()

# Agent: Coordination Specialist
## Task: Assess the flood situation at 123 Emergency St, New York, NY and coordinate 
                initial response efforts. Determine priority needs and available resources.
                Consider the following factors:
                1. Current flood severity
                2. Number of people affected
                3. Immediate safety concerns
# Agent: Emergency Response Specialist
## Task: Assess the current flood severity and the number of people affected at 123 Emergency St, New York, NY


# Agent: Emergency Response Specialist
## Final Answer: 
Based on the information gathered from the local government's emergency management office, current weather forecasts and flood severity reports, and satellite images, the flood at 123 Emergency St, New York, NY is classified as severe. Local emergency responders on the ground report that approximately 300 people are directly affected. Immediate assistance is required. I recommend deploying emergen